In [1]:
import pandas as pd
import numpy as np

# -----------------------
# Step 0: Load dataset
# -----------------------
file_path = r"C:\Users\rajat\OneDrive\Desktop\Inputs scorecard\merged_customer_data.xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1")

# -----------------------
# Step 1: Clean column names (strip spaces, lower, replace special chars)
# -----------------------
df.columns = df.columns.str.strip()       # remove leading and traling spaces
df.columns = df.columns.str.replace(" ", "_")  # replace spaces with underscore
df.columns = df.columns.str.replace("-", "_")  # replace dashes with underscore
df.columns = df.columns.str.strip()       # strip again in case
print("Cleaned Columns:", df.columns.tolist())

Cleaned Columns: ['Customer_ID', 'Age', 'Income_INR', 'Employment_Years', 'Marital_Status', 'Education_Level', 'Credit_History_Length', 'Outstanding_Loans', 'Loan_Amount', 'Loan_Tenure_Months', 'Savings_Account_Balance', 'Checking_Account_Balance', 'Pay_History', 'Delinquency_12M', 'Credit_Card_Utilization', 'Behavior_Spending_Score', 'Behavior_Repayment_Score', 'Default_x', 'No_of_Open_Accounts', 'No_of_Closed_Accounts', 'Total_Credit_Limit', 'Total_Current_Balance', 'Credit_Utilization_Ratio', 'No_of_Inquiries_6M', 'No_of_Inquiries_12M', 'DPD_30', 'DPD_60', 'DPD_90', 'Worst_Current_Status', 'Months_Since_Most_Recent_Delinquency', 'Max_Credit_Exposure', 'Oldest_Trade_Open_Months', 'Newest_Trade_Open_Months', 'Default_y']


In [2]:
# -----------------------
# Step 2: Drop ID + target duplicates
# -----------------------
# Drop only if columns exist (avoids KeyError)
drop_cols = [c for c in ["Customer_ID", "Default_x"] if c in df.columns]
df = df.drop(columns=drop_cols, errors="ignore")

target = "Default_y"   # main dependent variable
independent_vars = [col for col in df.columns if col != target]

# Separate numeric vs categorical
numeric_vars = df[independent_vars].select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_vars = df[independent_vars].select_dtypes(exclude=["int64", "float64"]).columns.tolist()

# -----------------------
# Step 2 Output Preview
# -----------------------
print("\n✅ Dataset after Step 2")
print("Shape:", df.shape)
print("\nFirst 5 rows:\n", df.head())
print("\nNumeric Variables:", numeric_vars)
print("\nCategorical Variables:", categorical_vars)


✅ Dataset after Step 2
Shape: (100, 32)

First 5 rows:
    Age  Income_INR  Employment_Years Marital_Status Education_Level  \
0   59     2309818                 0       Divorced        Graduate   
1   49     2431785                 8       Divorced    Postgraduate   
2   35      488998                27         Single        Graduate   
3   63     2043400                26         Single     High School   
4   28     1279111                 5       Divorced        Graduate   

   Credit_History_Length  Outstanding_Loans  Loan_Amount  Loan_Tenure_Months  \
0                      5                  9       448929                  48   
1                     12                  6       630466                  24   
2                     16                  9       623695                  72   
3                     16                  4       513556                  24   
4                      7                  2       625278                  24   

   Savings_Account_Balance  ...  No

In [3]:
# -----------------------
# Helper function: Calculate WOE & IV
# -----------------------

#✅ Weight of Evidence (WOE):
# It measures how well a feature separates good (non-default) and bad (default) cases by comparing their distributions across bins. 
# Higher separation = stronger predictor.

#✅ Information Value (IV):
# It quantifies the overall predictive power of a feature for the target variable.
#IV < 0.02 → Weak, 0.02–0.1 → Medium, 0.1–0.3 → Strong, >0.3 → Very Strong.

def calc_woe_iv(data, feature, target, bins=5, categorical=False):
    tmp = data.copy()

    # Numeric → binning
    if not categorical:
        tmp[feature] = pd.qcut(tmp[feature], q=bins, duplicates='drop')

    # Group by feature bins
    grouped = tmp.groupby(feature)[target].agg(['count', 'sum'])
    grouped = grouped.rename(columns={'sum': 'bad'})
    grouped['good'] = grouped['count'] - grouped['bad']

    # Avoid divide-by-zero
    grouped = grouped[(grouped['bad'] > 0) & (grouped['good'] > 0)]

    # Distribution of good & bad
    grouped['dist_good'] = grouped['good'] / grouped['good'].sum()
    grouped['dist_bad'] = grouped['bad'] / grouped['bad'].sum()

    # WOE - it computes the distribution of good vs. bad, then calculates WOE = ln(dist_good / dist_bad)
    grouped['woe'] = np.log(grouped['dist_good'] / grouped['dist_bad'])

    # IV - Finally, it computes IV = Σ((dist_good – dist_bad) × WOE)
    grouped['iv'] = (grouped['dist_good'] - grouped['dist_bad']) * grouped['woe']
    iv = grouped['iv'].sum()

    return grouped, iv



In [4]:
# -----------------------
# Step 3: Compute IV for all variables
# -----------------------
iv_summary = {}  # Dictionary to store variable → IV mapping

# Numeric variables (apply binning)
for col in numeric_vars:
    _, iv = calc_woe_iv(df, col, target, bins=5, categorical=False)
    iv_summary[col] = iv

# Categorical variables (no binning)
for col in categorical_vars:
    _, iv = calc_woe_iv(df, col, target, categorical=True)
    iv_summary[col] = iv

# -----------------------
# Step 4: Rank variables by IV
# -----------------------
# Convert dictionary to DataFrame
iv_summary = pd.DataFrame(list(iv_summary.items()), columns=["Variable", "IV"])

# Sort descending (higher IV = stronger predictor)
iv_summary = iv_summary.sort_values(by="IV", ascending=False).reset_index(drop=True)

print("📊 Information Value Summary (higher = stronger predictor):")
print(iv_summary)

# -----------------------
# Step 5: IV thresholds (rule of thumb)
# -----------------------
# IV < 0.02 → Useless
# 0.02–0.1 → Weak
# 0.1–0.3 → Medium
# 0.3–0.5 → Strong
# > 0.5 → Suspicious (may be overfitting / data leakage)


📊 Information Value Summary (higher = stronger predictor):
                                Variable        IV
0                        Education_Level  0.374709
1                Behavior_Spending_Score  0.308529
2                             Income_INR  0.286893
3                     Total_Credit_Limit  0.286893
4                                    Age  0.283594
5                     Loan_Tenure_Months  0.266022
6   Months_Since_Most_Recent_Delinquency  0.246311
7               Credit_Utilization_Ratio  0.227352
8                Credit_Card_Utilization  0.210313
9                       Employment_Years  0.193059
10              Behavior_Repayment_Score  0.167974
11                    No_of_Inquiries_6M  0.167789
12                 Credit_History_Length  0.161494
13                  Worst_Current_Status  0.157881
14                 No_of_Closed_Accounts  0.140442
15              Newest_Trade_Open_Months  0.123373
16              Checking_Account_Balance  0.116249
17                     

C:\Users\rajat\AppData\Local\Temp\ipykernel_23036\1194783680.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = tmp.groupby(feature)[target].agg(['count', 'sum'])
C:\Users\rajat\AppData\Local\Temp\ipykernel_23036\1194783680.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = tmp.groupby(feature)[target].agg(['count', 'sum'])
C:\Users\rajat\AppData\Local\Temp\ipykernel_23036\1194783680.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the fut

In [5]:
# -----------------------
#  Missing Values / Data Quality
# -----------------------

# Count missing values per column
missing_summary = df[independent_vars].isnull().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
print("\n🚨 Missing Values:\n", missing_summary)

# % of missing values
missing_pct = (df[independent_vars].isnull().mean() * 100).round(2)
print("\n% Missing by Column:\n", missing_pct[missing_pct > 0])

# 👉 Summary: This identifies columns with missing values.
# Features with too many missing values (e.g., >30%) may need imputation or removal.



🚨 Missing Values:
 Series([], dtype: int64)

% Missing by Column:
 Series([], dtype: float64)


In [6]:
# -------------------------------------------------------
# Multicollinearity Check using Correlation & VIF
# -------------------------------------------------------
# Summary: Correlation matrix shows pairwise relationships between numeric features.
#### A correlation matrix shows how strongly two variables are linearly related.
### Values range from -1 to +1:
### +1 → perfect positive relationship (as one increases, the other also increases).
### -1 → perfect negative relationship (as one increases, the other decreases).
### 0 → no linear relationship.


# VIF (Variance Inflation Factor) detects multicollinearity across features.
# Very high VIF means redundancy → may cause model instability.
# -------------------------------------------------------

from statsmodels.stats.outliers_influence import variance_inflation_factor  # Used to calculate VIF for numeric features

# 1. Correlation matrix of numeric features
corr_matrix = df[numeric_vars].corr()
print("🔗 Correlation Matrix:\n", corr_matrix)

# 2. Variance Inflation Factor (VIF) calculation
vif_data = pd.DataFrame()              # Create an empty DataFrame to store results
vif_data["Feature"] = numeric_vars     # Add feature names

# Calculate VIF for each feature
vif_data["VIF"] = [
    variance_inflation_factor(df[numeric_vars].dropna().values, i)
    for i in range(len(numeric_vars))
]

# Explanation:
# - For each feature index i:
#   1. Takes numeric feature data from df[numeric_vars]
#   2. Drops rows with missing values (dropna())
#   3. Converts DataFrame to NumPy array (.values)
#   4. Runs variance_inflation_factor(..., i), which:
#        • Regresses the i-th feature against all others
#        • Computes VIF = 1 / (1 - R²)

# 3. Display results sorted by highest VIF
print("\n📊 VIF Values:\n", vif_data.sort_values(by="VIF", ascending=False))


🔗 Correlation Matrix:
                                            Age  Income_INR  Employment_Years  \
Age                                   1.000000   -0.140334          0.085102   
Income_INR                           -0.140334    1.000000         -0.210966   
Employment_Years                      0.085102   -0.210966          1.000000   
Credit_History_Length                 0.093165   -0.098896          0.077525   
Outstanding_Loans                     0.032063    0.173999         -0.023823   
Loan_Amount                          -0.157384   -0.077044          0.084214   
Loan_Tenure_Months                   -0.010025   -0.167130         -0.035460   
Savings_Account_Balance               0.093598   -0.013674         -0.007163   
Checking_Account_Balance              0.049060   -0.164939         -0.161535   
Delinquency_12M                      -0.016795    0.036355         -0.011275   
Credit_Card_Utilization              -0.031438    0.005065         -0.002664   
Behavior_Spending

In [8]:
# -----------------------
# Step 6: Select final features
# -----------------------

# 1. Keep features with IV >= 0.02 (Weak or better)
selected_features = iv_summary[iv_summary["IV"] >= 0.02]["Variable"].tolist()

# 2. Drop features with too many missing values (>30% for example)
# missing_pct = (df[selected_features].isnull().mean() * 100)
# selected_features = [f for f in selected_features if missing_pct[f] <= 30]

# 3. Drop features with very high multicollinearity (VIF > 10)
# Recalculate VIF only on selected numeric variables
from statsmodels.stats.outliers_influence import variance_inflation_factor

numeric_selected = [f for f in selected_features if f in numeric_vars]
X_num = df[numeric_selected].dropna()

vif_data = pd.DataFrame()
vif_data["Feature"] = numeric_selected
vif_data["VIF"] = [variance_inflation_factor(X_num.values, i) 
                   for i in range(len(numeric_selected))]

# Drop high-VIF features
high_vif_features = vif_data[vif_data["VIF"] > 10]["Feature"].tolist()
selected_features = [f for f in selected_features if f not in high_vif_features]




In [10]:
# -----------------------
# Step 7: Final dataset
# -----------------------
final_df = df[selected_features + [target]].copy()

print("✅ Final Dataset Shape:", final_df.shape)
print("✅ Final Features:", selected_features)
print(final_df.head())

✅ Final Dataset Shape: (100, 28)
✅ Final Features: ['Education_Level', 'Behavior_Spending_Score', 'Income_INR', 'Total_Credit_Limit', 'Loan_Tenure_Months', 'Months_Since_Most_Recent_Delinquency', 'Credit_Utilization_Ratio', 'Credit_Card_Utilization', 'Employment_Years', 'Behavior_Repayment_Score', 'No_of_Inquiries_6M', 'Credit_History_Length', 'Worst_Current_Status', 'No_of_Closed_Accounts', 'Newest_Trade_Open_Months', 'Checking_Account_Balance', 'Pay_History', 'No_of_Inquiries_12M', 'Delinquency_12M', 'Loan_Amount', 'Outstanding_Loans', 'Oldest_Trade_Open_Months', 'Total_Current_Balance', 'Max_Credit_Exposure', 'No_of_Open_Accounts', 'DPD_30', 'Savings_Account_Balance']
  Education_Level  Behavior_Spending_Score  Income_INR  Total_Credit_Limit  \
0        Graduate                       73     2309818             1996830   
1    Postgraduate                       26     2431785             2664150   
2        Graduate                       66      488998             1618560   
3     Hi

In [11]:
# -----------------------
# Step 8: Save Final Dataset
# -----------------------

# Define save path (adjust as needed)
save_path = r"C:\Users\rajat\OneDrive\Desktop\Inputs scorecard\EDA.xlsx"

# Save to Excel
final_df.to_excel(save_path, index=False)

print(f"✅ Final dataset saved successfully at: {save_path}")


✅ Final dataset saved successfully at: C:\Users\rajat\OneDrive\Desktop\Inputs scorecard\EDA.xlsx


In [14]:
print("Total variables initially:", len(independent_vars))
print("After IV filtering:", len(iv_summary[iv_summary["IV"] >= 0.02]))



Total variables initially: 31
After IV filtering: 28
